# Consolidación de todos los años [2006-2024]

In [7]:
import pandas as pd
import os
import glob

# Ruta de la carpeta donde están los archivos sueltos
carpeta_procesados = os.path.join("..", "data", "processed")

# Lista de archivos CSV a procesar
archivos_csv = glob.glob(os.path.join(carpeta_procesados, "*.csv"))

# Diccionario para mapear nombres de columnas a un formato estándar
mapeo_columnas = {
    'Fecha': 'FECHA',
    'KITENI': 'KITENI',
    'Kiteni': 'KITENI',
    'PS-1': 'PS#1',
    'PS-2': 'PS#2',
    'PS-3': 'PS#3',
    'PS-4': 'PS#4',
    'Base Operativa Kiteni': 'KITENI'
}

# Crear una lista para almacenar los dataframes procesados
dataframes = []

# Procesar cada archivo CSV
for ruta_archivo in archivos_csv:
    try:
        # Leer el archivo CSV
        df = pd.read_csv(ruta_archivo)

        # Renombrar las columnas según el mapeo
        df.rename(columns=mapeo_columnas, inplace=True)

        # Eliminar las columnas 'Horas' y 'Num' si existen
        for columna in ['Horas', 'Num']:
            if columna in df.columns:
                df.drop(columns=[columna], inplace=True)

        # Asegurar que las columnas Año, Mes y Día estén presentes
        if 'FECHA' in df.columns:
            df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')

            if 'Año' not in df.columns:
                df['Año'] = df['FECHA'].dt.year
            if 'Mes' not in df.columns:
                df['Mes'] = df['FECHA'].dt.month
            if 'Día' not in df.columns:
                df['Día'] = df['FECHA'].dt.day

        # Rellenar FECHA usando Año, Mes y Día si está vacía
        if 'FECHA' not in df.columns or df['FECHA'].isnull().all():
            if {'Año', 'Mes', 'Día'}.issubset(df.columns):
                df['FECHA'] = pd.to_datetime(
                    df[['Año', 'Mes', 'Día']].astype(str).agg('-'.join, axis=1),
                    format='%Y-%m-%d',  # Especificar el formato explícitamente
                    errors='coerce'
                )

        # Asegurar que las columnas Año, Mes y Día sean enteros
        for columna in ['Año', 'Mes', 'Día']:
            if columna in df.columns:
                df[columna] = df[columna].fillna(0).astype(int)

        # Añadir el dataframe procesado a la lista
        dataframes.append(df)
        print(f"Archivo procesado: {os.path.basename(ruta_archivo)}")
    except Exception as e:
        print(f"Error procesando el archivo {ruta_archivo}: {e}")

# Consolidar todos los dataframes
if dataframes:
    try:
        # Concatenar todos los DataFrames
        df_consolidado = pd.concat(dataframes, ignore_index=True)

        # Asegurar que Año, Mes y Día sean enteros en el consolidado final
        for columna in ['Año', 'Mes', 'Día']:
            if columna in df_consolidado.columns:
                df_consolidado[columna] = df_consolidado[columna].fillna(0).astype(int)

        # Generar la columna FECHA si aún no está completa
        if 'FECHA' not in df_consolidado.columns or df_consolidado['FECHA'].isnull().any():
            df_consolidado['FECHA'] = pd.to_datetime(
                df_consolidado[['Año', 'Mes', 'Día']].astype(str).agg('-'.join, axis=1),
                format='%Y-%m-%d',  # Especificar el formato explícitamente
                errors='coerce'
            )

        # Guardar el archivo consolidado
        ruta_salida = os.path.join(carpeta_procesados, "data_consolidado.csv")
        df_consolidado.to_csv(ruta_salida, index=False)
        print(f"Archivo consolidado guardado en: {ruta_salida}")
    except Exception as e:
        print(f"Error procesando el archivo consolidado: {e}")
else:
    print("No se encontraron datos para consolidar.")


Archivo procesado: data2006-2010.csv
Archivo procesado: data2011-2015.csv
Archivo procesado: data2016-2019.csv
Archivo procesado: data2020-2024.csv
Archivo procesado: data_consolidado.csv
Archivo consolidado guardado en: ..\data\processed\data_consolidado.csv


In [8]:

# Recuento por año
if 'Año' in df_consolidado.columns:
    recuento_por_anio = df_consolidado.groupby('Año').size().reset_index(name='Recuento')
    print("Recuento de filas por año:")
    print(recuento_por_anio)
else:
    print("La columna 'Año' no existe en el DataFrame consolidado.")



Recuento de filas por año:
     Año  Recuento
0      0      1471
1   2006      1825
2   2007      1092
3   2008      1830
4   2009      1825
5   2010      1095
6   2011      1825
7   2012      1830
8   2013      1825
9   2014      1825
10  2015      1825
11  2016      1830
12  2017      1825
13  2018      1825
14  2019      1825
15  2020      1830
16  2021      1773
17  2022       465
18  2023      1860
19  2024      1675


In [9]:
# Crear un rango completo de fechas desde la mínima hasta la máxima en el DataFrame
rango_fechas = pd.date_range(start=df_consolidado['FECHA'].min(), end=df_consolidado['FECHA'].max())

# Detectar fechas reales
fechas_reales = set(df_consolidado['FECHA'].dropna())

# Detectar fechas faltantes
fechas_faltantes = sorted(set(rango_fechas) - fechas_reales)

# Crear un DataFrame de fechas faltantes
fechas_faltantes_df = pd.DataFrame(fechas_faltantes, columns=['FECHA'])
fechas_faltantes_df['Año'] = fechas_faltantes_df['FECHA'].dt.year
fechas_faltantes_df['Mes'] = fechas_faltantes_df['FECHA'].dt.month

# Agrupar por año y mes para obtener el recuento de faltantes
recuento_por_anio_mes = fechas_faltantes_df.groupby(['Año', 'Mes']).size().reset_index(name='Faltantes')

# Mostrar el recuento
print("Recuento de fechas faltantes por año y mes:")
print(recuento_por_anio_mes)


Recuento de fechas faltantes por año y mes:
     Año  Mes  Faltantes
0   2007    7          1
1   2010    5          1
2   2010    6          2
3   2016   12         30
4   2017   12         30
5   2022    1         31
6   2022    2         28
7   2022    3         31
8   2022    4         30
9   2022    5         31
10  2022    6         30
11  2022    7         31
12  2022    8         31
13  2022    9         30
